#  EMF RBM Omniglot

### Binarized, 20 Epochs, faster learning rate

Look at different ways of computing the

- energy density
- free energy density
- entropy density

We can

- use the TAP S and MF Energy *U_naive   
    - which takes an equlibration step first  
    - S(mv, mh), U_naive(mv, mh)
- use a simple h = means_hiddens() step
    - S(v, h), E_MF(v, h)


In [ ]:
import numpy as np
import h5py

import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 

from sklearn import linear_model, datasets, metrics, preprocessing 
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

import psutil

In [ ]:
from sklearn.datasets import fetch_mldata
from sklearn.utils.validation import assert_all_finite
from scipy.sparse import csc_matrix, csr_matrix, lil_matrix
from sklearn.preprocessing import Binarizer
np.seterr(all='warn')

In [ ]:
from scipy.io import loadmat
import matplotlib.pyplot as plt
import matplotlib as mpl

def show_image(image): 
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(image, cmap=mpl.cm.Greys)
    imgplot.set_interpolation('nearest')
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')
    plt.show()

In [ ]:
import rbm_datasets

In [ ]:
train_x, train_y, train_char, test_x, test_y, test_char = rbm_datasets.load_omniglot_iwae()

In [ ]:
from sklearn.preprocessing import binarize
X_train = binarize(train_x, threshold=0.5, copy=True)

In [ ]:
for i in range(5):
    show_image(X_train[i+100].reshape(28,28))
    show_image(train_x[i+100].reshape(28,28))

In [ ]:
X_train.shape

### EMF RBM

In [ ]:
psutil.swap_memory()

### mean Entropy, Energy, and Free Energy

In [ ]:
import emf_rbm
rbm = emf_rbm.EMF_RBM(verbose=True, monitor=True, learning_rate=0.1, n_iter=20)
print rbm
rbm= rbm.fit(X_train)

In [ ]:
plt.plot(rbm.entropies-rbm.entropies[0], color='red', label='Entropy')
plt.plot(rbm.mean_field_energies-rbm.mean_field_energies[0], color='blue', label='MF Energy')
plt.plot(rbm.free_energies-rbm.free_energies[0], color='green', label='Free Energy')
plt.legend(loc='upper right')
plt.title("Ominglot: Free Energy, Mean Field Enegry, & Entropy vs Epochs")


In [ ]:
from sklearn.preprocessing import binarize
X_test = binarize(test_x, threshold=0.5, copy=True)

In [ ]:
np.unique(train_y)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

F_train = rbm.transform(X_train)
F_test = rbm.transform(X_test)

for c in [0.01, 0.1, 1, 10, 100]:
    classifier = LinearSVC(c=C)
    classifier.fit(F_train, train_y)
    test_emf_rbm_pred_y = classifier.predict(F_test)
    emf_accuracy = accuracy_score(y_pred=test_emf_rbm_pred_y, y_true=test_y)
    print c, emf_accuracy